In [1]:
import pandas as pd
import os

from mbdvv import app, get_solids

In [2]:
solid_data = get_solids(app.ctx)

In [3]:
solid_data['atoms'].head()

ene
Z  atom full_conf   conf              
46 Pd   [Kr]4d10    -     -5045.785878
22 Ti   [Ar]4s2,3d2 -2,-1          NaN
                    -2,0   -853.611395
                    -2,1           NaN
                    -2,2           NaN

In [4]:
solid_data['solids'].head()

,label,scale,data
0,crystals/Ag/0.97,0.97,"{'coords': {'unit': 'a.u.', 'value': [[0.], [0..."
1,crystals/Ag/0.98,0.98,"{'coords': {'unit': 'a.u.', 'value': [[0.], [0..."
2,crystals/Ag/0.99,0.99,"{'coords': {'unit': 'a.u.', 'value': [[0.], [0..."
3,crystals/Ag/1.0,1.00,"{'coords': {'unit': 'a.u.', 'value': [[0.], [0..."
4,crystals/Ag/1.01,1.01,"{'coords': {'unit': 'a.u.', 'value': [[0.], [0..."
